In [1]:
import stockutils
import pandas_datareader.data as web, re, sqlite3, datetime

In [2]:
class StockData:
    sql_path = 'data/completecompanylist.sql'
    sql_table_name = 'companytable'
    def __init__(self, se = None, update = False):
        self.company_table = stockutils.company_table(se = se, update = update)
        self.se = se
   
    def lookup_ticker(self, company_name, sql = True, lookup = False):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        if isinstance(company_name,str):
            company_name = [company_name]
        result_table, conn = [], None 
        if not sql:
            for name in company_name:
                for name in [name, name.upper()]:
                    query = r".*" + r".*".join(list(name)) + r".*"
                    for row in self.company_table.keys():
                        match = re.search(query,row)
                        if match:
                            ticker = self.lookup_ticker(match.group(), lookup = True)
                            if ticker:
                                result_table.extend(ticker)
        else:
            conn = sqlite3.connect(self.sql_path)
            c = conn.cursor()
            for name in company_name:
                if self.se: 
                    query = (name, self.se,)
                    rows = c.execute('SELECT symbol,name,se FROM ' + self.sql_table_name + ' WHERE name LIKE ? AND se = ?;', query).fetchall()
                else:
                    query = (name,)
                    rows = c.execute('SELECT symbol,name,se FROM ' + self.sql_table_name + ' WHERE name LIKE ?;', query).fetchall()
                if rows:
                    result_table.extend(rows)
                else:
                    if not lookup:
                        print('Switching to regular expressions for ' + name + ' lookup.')
                        found = self.lookup_ticker(name, sql = False)
                        if found:
                            for item in found:
                                result_table.append(item)
                        else:
                            result_table.append(('N/A', name))
            conn.close()
        return result_table
        
    def ipo_year(self,company_name, sql = True):
        if isinstance(company_name,str):
            company_name = [company_name]
        list_years = []
        conn = sqlite3.connect(self.sql_path)
        c = conn.cursor()
        for name in company_name:
            query = (name, )
            list_years.extend(c.execute('SELECT ipoyear, symbol FROM ' + self.sql_table_name + ' WHERE name LIKE ?;', query).fetchall())
        return list_years
            
#       def stock_price(self, ticker, date):
            
        
        

In [3]:
#update
w = StockData(update = True)

In [4]:
w = StockData()

In [5]:
x = StockData('nasdaq')

In [6]:
y = StockData('amex')

In [7]:
z = StockData('nyse')

In [8]:
x.lookup_ticker('NVIDIA Corporation')

[('NVDA', 'NVIDIA Corporation', 'nasdaq')]

In [9]:
x.lookup_ticker(['Intel Corporation', 'Nvidia', 'Apple', 'Microsoft', 'Becton Dickinson'])

Switching to regular expressions for Nvidia lookup.
Switching to regular expressions for Apple lookup.
Switching to regular expressions for Microsoft lookup.
Switching to regular expressions for Becton Dickinson lookup.


[('INTC', 'Intel Corporation', 'nasdaq'),
 ('NVDA', 'NVIDIA Corporation', 'nasdaq'),
 ('ARCI', 'Appliance Recycling Centers of America, Inc.', 'nasdaq'),
 ('AAOI', 'Applied Optoelectronics, Inc.', 'nasdaq'),
 ('AMCC', 'Applied Micro Circuits Corporation', 'nasdaq'),
 ('APDN', 'Applied DNA Sciences Inc', 'nasdaq'),
 ('AGTC', 'Applied Genetic Technologies Corporation', 'nasdaq'),
 ('AMAT', 'Applied Materials, Inc.', 'nasdaq'),
 ('AAPL', 'Apple Inc.', 'nasdaq'),
 ('MSFT', 'Microsoft Corporation', 'nasdaq'),
 ('N/A', 'Becton Dickinson')]

In [10]:
z.lookup_ticker("Dick's")

Switching to regular expressions for Dick's lookup.


[('DKS', "Dick's Sporting Goods Inc", 'nyse')]

In [11]:
z.lookup_ticker('Becton')

Switching to regular expressions for Becton lookup.


[('BGE^B', 'Baltimore Gas & Electric Company', 'nyse'),
 ('BIT', 'BlackRock Multi-Sector Income Trust', 'nyse'),
 ('INF', 'Brookfield Global Listed Infrastructure Income Fund', 'nyse'),
 ('BPI', 'Bridgepoint Education, Inc.', 'nyse'),
 ('WAB', 'Westinghouse Air Brake Technologies Corporation', 'nyse'),
 ('BAC', 'Bank of America Corporation', 'nyse'),
 ('BSX', 'Boston Scientific Corporation', 'nyse'),
 ('BNED', 'Barnes & Noble Education, Inc', 'nyse'),
 ('IBM', 'International Business Machines Corporation', 'nyse'),
 ('BHE', 'Benchmark Electronics, Inc.', 'nyse'),
 ('BR', 'Broadridge Financial Solutions, Inc.', 'nyse'),
 ('JBT', 'John Bean Technologies Corporation', 'nyse'),
 ('BTZ', 'BlackRock Credit Allocation Income Trust', 'nyse'),
 ('BDX', 'Becton, Dickinson and Company', 'nyse')]

In [12]:
w.lookup_ticker(['Intel', 'BD'])

Switching to regular expressions for Intel lookup.
Switching to regular expressions for BD lookup.


[('SHI', 'SINOPEC Shangai Petrochemical Company, Ltd.', 'nyse'),
 ('CAI', 'CAI International, Inc.', 'nyse'),
 ('IPG', 'Interpublic Group of Companies, Inc. (The)', 'nyse'),
 ('GPIC', 'Gaming Partners International Corporation', 'nasdaq'),
 ('ICFI', 'ICF International, Inc.', 'nasdaq'),
 ('BAF', 'BlackRock Income Investment Quality Trust', 'nyse'),
 ('HIL', 'Hill International, Inc.', 'nyse'),
 ('VKI', 'Invesco Advantage Municipal Income Trust II', 'amex'),
 ('NUTR', 'Nutraceutical International Corporation', 'nasdaq'),
 ('ATNI', 'ATN International, Inc.', 'nasdaq'),
 ('IPKW', 'PowerShares International BuyBack Achievers Portfolio', 'nasdaq'),
 ('CHH', 'Choice Hotels International, Inc.', 'nyse'),
 ('WFT', 'Weatherford International plc', 'nyse'),
 ('REXR', 'Rexford Industrial Realty, Inc.', 'nyse'),
 ('VFL', 'Delaware Investments Florida Insured Municipal Income Fund', 'amex'),
 ('GOL', 'Gol Linhas Aereas Inteligentes S.A.', 'nyse'),
 ('PNQI', 'PowerShares Nasdaq Internet Portfolio', 

In [13]:
x.ipo_year('Intel Corporation')

[('n/a', 'INTC')]

In [14]:
z.ipo_year(['Blackrock Core Bond Trust', 'BlackRock Credit Allocation Income Trust'])

[('2001', 'BHK'), ('2006', 'BTZ')]